# Accelerating Text-to-SQL Inference on Vanna with NVIDIA NIM
This notebook demonstrates how to optimize Vanna’s open-source text-to-SQL pipeline using NVIDIA NIM and NeMo Retriever for faster and more efficient analytics.

## 1. Prerequisites
Make sure to install the following:
- Python 3.8+
- `vanna`, `langchain`, `milvus`, `openai`, `nvidia-langchain`,`pandas`, `kagglehub`,`numpy`  
- Access to NVIDIA NIM endpoints

In [ ]:
%pip install "pymilvus[model]"
%pip install --upgrade --quiet  langchain-nvidia-ai-endpoints
%pip install vanna
%pip install kagglehub
%pip install pandas
%pip install numpy
%pip install sqlite3
%pip install openai

## 2. Data Preparation
We use two Steam datasets from Kaggle - [Steam Games Dataset](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data) and [Games on Steam](https://www.kaggle.com/datasets/sujaykapadnis/games-on-steam/). We follow the preprocess steps mentioned in the [excellent Kaggle notebook](https://www.kaggle.com/code/terencicp/steam-games-data-transformation). After preprocessing, three CSVs will be generated:
 - `tableau_games.csv`
 - `tableau_categories.csv`
 - `tableau_tags.csv`

 You can follow the steps outlined below or directly use the the processed data in `processed_dataset` folder and jump to Step 3.

### 2.1 Download Steam Datasets

In [ ]:
import kagglehub

# Download first dataset
first_dataset_path = kagglehub.dataset_download("fronkongames/steam-games-dataset")

print("Path to dataset files:", first_dataset_path)

# Download second dataset
second_dataset_path = kagglehub.dataset_download("sujaykapadnis/games-on-steam")

print("Path to dataset files:", second_dataset_path)

### 2.2 Preprocess first dataset

In [ ]:
import json

first_file = first_dataset_path + '/games.json'
with open(first_file, 'r') as file:
    json_data = json.load(file)

In [ ]:
import pandas as pd

# Remove unnecessary variables
unnecessary_vars = [
    'packages', 'screenshots', 'movies', 'score_rank', 'header_image',
    'reviews', 'website', 'support_url', 'notes', 'support_email',
    'recommendations', 'user_score', 'median_playtime_forever',
    'median_playtime_2weeks', 'required_age', 'metacritic_score',
    'metacritic_url', 'peak_ccu', 'detailed_description', 'about_the_game',
    'windows', 'mac', 'linux', 'achievements', 'full_audio_languages',
    'genres', 'dlc_count', 'supported_languages', 'developers',
    'publishers', 'average_playtime_forever', 'average_playtime_2weeks',
    'discount'
]

# Process each game's information and store in a list
games = [{
    **{k: v for k, v in game_info.items() if k not in unnecessary_vars},
    'tags': list(tags.keys()) if isinstance((tags := game_info.get('tags', {})), dict) else [],
    'tag_frequencies': list(tags.values()) if isinstance(tags, dict) else [],
    'app_id': app_id
} for app_id, game_info in json_data.items()]

# Create a DataFrame from the processed list
df = pd.DataFrame(games)

Great! We've got the data we need, but it seems we have more rows than there are games on Steam.

Some games just seem to be developer tests. Let's remove them. We'll also remove games with no reviews or no categories

In [ ]:
# Filter games without sales, reviews or categories
df2 = df[~((df['estimated_owners'] == "0 - 0") | (df['positive'] + df['negative'] == 0) | (df['categories'].str.len() == 0))]

To keep things simple, we will also remove games older than 2013, since there are very few games more than 10 year old on Steam:

In [ ]:
# Filter games released before 2013
df2 = df2.copy()
df2['release_date'] = pd.to_datetime(df2['release_date'], format='mixed')
df2 = df2[df2['release_date'].dt.year >= 2013]

Next, split the 'estimated_owners' column into two different variables.

In [ ]:
# Split estimated_owners into two: min_owners and max_owners
df2[['min_owners', 'max_owners']] = df2['estimated_owners'].str.split(' - ', expand=True)

# Remove the original field
df2 = df2.drop('estimated_owners', axis=1)

Delete the outlier game with more than $800 price

In [ ]:
# Remove games with price > $800
df2 = df2[df2['price'] <= 800]


### 2.3 Merge the two datasets

The second dataset we'll use contains information about game duration.
Let's read the second dataset


In [ ]:
second_file = second_dataset_path + '/steamdb.json'
df_second_dataset = pd.read_json(second_file)

We're only interested in the column 'hltb_single', that contains the information on game length we need. Each Steam game has a unique identifier we can use to join the data from both datasets. This unique identifier is found in the column 'app_id' of the first dataset, and in the column 'sid' of the second dataset. First we'll have to convert 'app_id' to integer since it is currently an object. Let's join the data and see the result:

In [ ]:
# Convert 'app_id' integer
df2['app_id'] = pd.to_numeric(df2['app_id'], errors='coerce').astype('Int64')

# Perform a left join for 'hltb_single'
df_merged = pd.merge(df2, df_second_dataset[['sid', 'hltb_single']], left_on='app_id', right_on='sid', how='left')

# Drop the redundant 'sid' column
df_merged.drop('sid', axis=1, inplace=True)

Some games are extreme outliers in terms of duration. This is not caused by these games being extremely long but by the fact that some games can be played indefinitely and very few users have reported game length for these types of games. This might distort our analysis, so we'll limit the maximum duration of games at 100 hours, which is a reasonable upper limit for most games:

In [ ]:
# Limit game duration to 100 hours
df_merged['hltb_single'] = df_merged['hltb_single'].apply(lambda x: 100 if x > 100 else x)

### 2.4 Normalizing data

The DataFrame contains fields such as 'categories' and 'tags' that consist of lists of values. To normalize the data for storage in a SQL database, we need to break these fields into separate tables. Each table will maintain a relationship with the main table through the 'app_id' foreign key, following standard database normalization practices.

In [ ]:
# Create a separate DataFrame for each list-type column
df_categories = df_merged.explode('categories')[['app_id', 'categories']]
df_tags = df_merged.explode('tags')[['app_id', 'tags']]
df_frequencies = df_merged.explode('tag_frequencies')['tag_frequencies']
df_tags['tag_frequencies'] = df_frequencies.values

# Remove the list columns from the main DataFrame
columns_to_remove = ['categories', 'tags', 'tag_frequencies']
df_imploded = df_merged.drop(columns=columns_to_remove)

The main DataFrame is ready. We will remove any categories and tags with less than 50 games, since they are not relevant enough.

In [ ]:
# Filter out categories with less than 50 games
categories_counts = df_categories['categories'].value_counts()
categories_to_keep = categories_counts[categories_counts >= 50].index.tolist()
df_categories = df_categories[df_categories['categories'].isin(categories_to_keep)]

In [ ]:
# Filter out tags with less than 50 games
tags_counts = df_tags['tags'].value_counts()
tags_to_keep = tags_counts[tags_counts >= 50].index.tolist()
df_tags = df_tags[df_tags['tags'].isin(tags_to_keep)]

### 2.5 Save preprocessing results as CSV

Finally, we'll save the results as CSV files that we'll ingest into the SQL Database.

In [ ]:
df_imploded.to_csv('./processed_dataset/games.csv', index=False)
df_categories.to_csv('./processed_dataset/categories.csv', index=False)
df_tags.to_csv('./processed_dataset/tags.csv', index=False)

## 3. Setting Up Vanna with NVIDIA NIM and NeMo Retriever

First, set the NVIDIA API Key. If you don't  find it [here](https://build.nvidia.com/)

In [ ]:
nvidia_api_key = '...'

### Define Vanna Class using Milvus and OpenAI 

In [ ]:
from pymilvus import MilvusClient, model
from vanna.milvus import Milvus_VectorStore
from vanna.openai import OpenAI_Chat
from openai import OpenAI

class VannaMilvus(Milvus_VectorStore, OpenAI_Chat):
    def __init__(self, llm_client, config=None):
        Milvus_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, client=llm_client, config=config)

### Create NIM Client based on OpenAI Wrapper

In [ ]:
def get_openai_client():
    client = OpenAI(
        base_url = "https://integrate.api.nvidia.com/v1",
        api_key = nvidia_api_key
    )
    return client

llm_client = get_openai_client()

### Create Nvidia Embedder (Langchain)

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
import numpy as np

nvidia_embedder = NVIDIAEmbeddings(
    model="nvidia/llama-3.2-nv-embedqa-1b-v2",
    base_url='https://integrate.api.nvidia.com/v1',
    api_key=nvidia_api_key
)

class EmbeddingWrapper:
    def __init__(self, embedder):
        self.embedder = embedder

    def encode_documents(self, texts):
        result = self.embedder.embed_documents(texts)
        return [np.array(r) for r in result]

    def encode_queries(self, texts):
        embeddings = []
        for text in texts:
            embeddings.append(self.embedder.embed_query(text))
        return embeddings

vanna_embedder = EmbeddingWrapper(nvidia_embedder)


### Define the Vector DB Client
To keep things simple, we will use a local Milvus vector DB.

In [ ]:
milvus_uri = "./milvus_nvidia.db"
milvus_client_nvidia = MilvusClient(uri=milvus_uri)

### Create a Vanna instance using the LLM, Embedder and the Vector DB defined above

In [ ]:
# Specify the LLM used for SQL generation
model_name = "meta/llama-3.1-70b-instruct"

# Define the configuration for Vanna instance
config_nvidia = {
        "model": model_name,
        "milvus_client": milvus_client_nvidia,
        "embedding_function": vanna_embedder,
        "n_results": 2,  # The number of results to return from Milvus semantic search.
    }

vn_nvidia = VannaMilvus(llm_client, config=config_nvidia)

## 4. Ingest processed Steam data into a SQL DB

In [ ]:
import sqlite3
import pandas as pd

# Specify the path to the SQLite database
sqlite_path = 'steam_data.db'

# Connect to the SQLite database
sql_connect = sqlite3.connect(sqlite_path)
c = sql_connect.cursor()

# Create tables
init_sqls = """
CREATE TABLE IF NOT EXISTS games (
    app_id INTEGER PRIMARY KEY,
    name TEXT,
    release_date TEXT,
    price REAL,
    short_description TEXT,
    positive INTEGER,
    negative INTEGER,
    min_owners INTEGER,
    max_owners INTEGER,
    hltb_single REAL
);

CREATE TABLE IF NOT EXISTS categories (
    app_id INTEGER,
    categories TEXT,
    FOREIGN KEY (app_id) REFERENCES games(app_id)
);

CREATE TABLE IF NOT EXISTS tags (
    app_id INTEGER,
    tags TEXT,
    tag_frequencies TEXT,
    FOREIGN KEY (app_id) REFERENCES games(app_id)
);
"""

for sql in init_sqls.split(";"):
    c.execute(sql)

# Read the CSV files
games_df = pd.read_csv('processed_dataset/games.csv')
categories_df = pd.read_csv('processed_dataset/categories.csv')
tags_df = pd.read_csv('processed_dataset/tags.csv')

# Insert data into tables
games_df.to_sql('games', sql_connect, if_exists='append', index=False)
categories_df.to_sql('categories', sql_connect, if_exists='append', index=False)
tags_df.to_sql('tags', sql_connect, if_exists='append', index=False)

sql_connect.commit()



Great, the data is ingested in the SQL DB. Now, let's connect the SQL DB to Vanna and start giving more context on our data.

In [ ]:
# Connect to the SQLite database
vn_nvidia.connect_to_sqlite(sqlite_path)

## 5. Let's train Vanna on our dataset

In [ ]:
# Remove existing training data
existing_training_data = vn_nvidia.get_training_data()
if len(existing_training_data) > 0:
    for _, training_data in existing_training_data.iterrows():
        vn_nvidia.remove_training_data(training_data["id"])

# Get the DDL of the SQLite database
df_ddl = vn_nvidia.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

# Train the model on the DDL data
for ddl in df_ddl["sql"].to_list():
    vn_nvidia.train(ddl=ddl)

In [ ]:
# Add documentation about your business terminology or definitions.
vn_nvidia.train(
    documentation="""
    This dataset is used to answer questions about the game trends.
    """
)

# Add documentation about the tables
vn_nvidia.train(
    documentation="""
    The games table contains information about the games.
    The app_id is the unique identifier for the game.This is a primary key.
    The name is the name of the game.
    The release_date is the date the game was released.
    The price is the price of the game. Price in USD, 0.0 if its free.
    The short_description is a brief description of the game.
    The positive is the number of positive reviews or votes.
    The negative is the number of negative reviews or votes.
    The min_owners is the minimum number of owners. Used together with max_owners to get an estimate of the player base.
    The max_owners is the maximum number of owners. Used together with min_owners to get an estimate of the player base.
    The hltb_single is the average playtime of the game. This is an estimate.
    """
)

vn_nvidia.train(
    documentation="""
    The categories table contains information about the categories of the games.
    The app_id is the unique identifier for the game.
    The categories is the categories of the game.
    The app_id is a foreign key to the games table.
    """
)

vn_nvidia.train(
    documentation="""
    The tags table contains information about the tags of the games.
    The app_id is the unique identifier for the game.
    The tags is the tags of the game. These are user defined.
    The tag_frequencies is the frequencies of the tags.
    The app_id is a foreign key to the games table.
    """
)

Let's verify the training data once.

In [ ]:
training_data = vn_nvidia.get_training_data()
training_data

## 6. Let's ask questions

In [ ]:
sql = vn_nvidia.generate_sql("Which 5 games have the most positive reviews and how many?")
vn_nvidia.run_sql(sql)

In [ ]:
sql = vn_nvidia.generate_sql("Which indie game has the biggest player base?")
vn_nvidia.run_sql(sql)

In [ ]:
sql = vn_nvidia.generate_sql("Which category has the maximum number of gamers and how many?")
vn_nvidia.run_sql(sql)